In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

#building image data generator

import keras
import tensorflow


from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions



In [ ]:
#load our img to data generator
train_datagen = ImageDataGenerator(zoom_range=0.5, shear_range=0.3, horizontal_flip=True, preprocessing_function= preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function= preprocess_input)

In [ ]:
#pass the path
train = train_datagen.flow_from_directory(directory= "/content/drive/MyDrive/Plant leaf disease detection/Image Data base",
                                          target_size= (256,256),
                                          batch_size=32)
val = val_datagen.flow_from_directory(directory= "/content/drive/MyDrive/Plant leaf disease detection/val",
                                          target_size= (256,256),
                                          batch_size=32)

Found 20999 images belonging to 26 classes.
Found 3686 images belonging to 26 classes.


In [ ]:
t_img , label = train.next()

In [ ]:
def plotImage(img_arr, label):

     for im , l in zip(img_arr , label):
       plt.figure(figsize=(5,5))
       plt.imshow(im)
       plt.show()

In [ ]:
plotImage(t_img[:3], label[:3])

Building our model

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
import keras
import sys

In [ ]:
base_model = VGG19(input_shape=(256,256,3), include_top=False)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
x = Flatten()(base_model.output)

x = Dense(units = 38, activation='softmax')(x)

#createing our model
model = Model(base_model.input, x)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer= 'adam' , loss= keras.losses.categorical_crossentropy, metrics=['accuracy'])

Early Stopping and Model Check point

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

#early stopping
es = EarlyStopping(monitor= 'val_accuracy', min_delta= 0.01, patience= 3, verbose=1)

#model check point
mc = ModelCheckpoint(filepath="best_model.h5",
                     monitor= 'val_accuracy',
                     min_delta= 0.01,
                     patience= 3,
                     verbose=1,
                     save_best_only= True)

cb = [es, mc]

In [ ]:
his = model.fit_generator(train,
                          steps_per_epoch= 13,
                          epochs= 20,
                          verbose=1,
                          callbacks= cb,
                          validation_data= val,
                          validation_steps=  13)

In [ ]:
h= his.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c="red")
plt.title("acc vs v-acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c="red")
plt.title("loss vs v-loss")
plt.show()

In [ ]:
#load best model

from keras.models import load_model

model = load_model("/content/best_model.h5")

In [ ]:
acc= model.evaluate_generator(val)[1]

print(f" the accuracy of your model is = {acc*100} %")

In [ ]:
ref = dict(zip(list(train.class_indices.values()) , list(train.class_indices.keys())))

In [ ]:
#will give the test img and will see if they are able to predict the correct output

def prediction(path):
     img = load_img(path, target_size= (256,256))

     i = img_to_array(img)

     im = preprocess_input(i)

     im = np.expand_dims(im, axis= 0)

     print(img.shape)

     pred = np.argmax(model.predict(img))

     print(f" the img belongs to {ref[pred]} ")

In [ ]:
path = "/content/drive/MyDrive/Plant leaf disease detection/test/Anthracnose_in_tea1.jpg"

prediction(path)